
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
# создадим переменные по условию
experiment_yawn = 10     # 10 Испытуемых зевнули
experiment_all = 34      # 34 Всего в экспериментальной группе
control_yawn = 4         # 4 из контрольной группы зевнули
control_all = 16         # 16 - объем контрольной выборки

In [2]:
# Будем считать, что выборки независимы, тогда рассмотрим Т-критерий для бернулли

In [3]:
import scipy.stats

In [4]:
# введем функцию для расчета p - value
from numpy import sqrt
import scipy.stats as st
def count_p_value(sample1, n1, sample2, n2):
    p1 = sample1 / n1
    p2 = sample2 / n2
    P = (p1 * n1 + p2 * n2) / (n1 + n2)
    z_point = (p1 - p2) / sqrt( P * (1 - P) * (1 / n1 + 1 / n2)  ) 
    return 1 - st.norm.cdf(z_point)

In [5]:
res = count_p_value(experiment_yawn, experiment_all, control_yawn, control_all)

In [6]:
print('{:.4f}'.format(res))

0.3729


Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими,и сто — поддельными. На рисунке ниже показаны измеренные признаки:

<img src="back_horizontal.jpg">

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:
* логистическая регрессия по признакам $X_{1}$, $X_{2}$, $X_{3}$
* логистическая регрессия по признакам $X_{4}$, $X_{5}$, $X_{6}$

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5 * 10^{-8}$, нужно ввести 8).



In [7]:
# импортируем модули 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
# загрузим данные и визуализируем их
data = pd.read_csv('banknotes.txt', delimiter='\t')

In [9]:
data.head(5)

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [10]:
# Отделим признаки на две группы: {𝑋1 ,  𝑋2 ,  𝑋3}, {𝑋4 ,  𝑋5 ,  𝑋6}
first_group_features = ['X1', 'X2', 'X3']
second_group_features = ['X4', 'X5', 'X6']

In [11]:
labels = data['real']

In [12]:
labels.shape

(200,)

In [13]:
# Разделим на обучение и тест с помощь метода train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, labels,test_size = 0.25, random_state = 1)

In [14]:
print(train_data.shape)

(150, 7)


## Оказывается, что необходимо указывать liblinear в параметр solver, так как дефолтное значение изменилось в более современных версиях sklearn

In [15]:
# Обучим модели
model_1 = LogisticRegression(solver = 'liblinear')

model_1.fit(train_data[first_group_features], train_labels)

residual_1 =  abs(test_labels - model_1.predict(test_data[first_group_features]))

In [16]:
model_2 = LogisticRegression(solver = 'liblinear')
model_2.fit(train_data[second_group_features], train_labels)
residual_2 = abs(test_labels - model_2.predict(test_data[second_group_features]))

In [17]:
# Так как выборки зависимы, то необходимо использовать совместные значения
import scipy.stats as st
from numpy import sqrt
def count_p_value_rel(sample1, sample2, alternative = 'two-sided'):
    g,f = count_rel_matrix_elems(sample1, sample2)
    z = float(f - g) / sqrt(f + g - float((f - g) ** 2) / len(sample1))
    if (alternative == 'two-sided'):
        return 2 * (1 - st.norm.cdf(abs(z)))
    elif (alternative == 'more'):
        return 1 - st.norm.cdf(z)
    elif (alternative == 'less'):
        return stats.norm.cdf(z)
    else:
        raise ValueError('wrong alternative')

In [18]:
def count_rel_matrix_elems(sample1, sample2):
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in zip(sample1, sample2)])
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in zip(sample1, sample2)])
    return (g,f)

In [19]:
print('{:.4f}'.format(count_p_value_rel(residual_1, residual_2)))

0.0033


In [20]:
def conf_int(sample1, sample2, alpha = 0.05):
    g,f = count_rel_matrix_elems(sample1, sample2)
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    lower_bound = (f - g) / len(sample1) - z * sqrt( (f + g) / len(sample1) ** 2  - (f - g) ** 2 / len(sample1) ** 3)
    upper_bound = (f - g) / len(sample1) + z * sqrt( (f + g) / len(sample1) ** 2  - (f - g) ** 2 / len(sample1) ** 3)
    return (lower_bound, upper_bound)

In [21]:
print('[%.4f, %.4f]' % conf_int(residual_1, residual_2, alpha=0.05))

[0.0599, 0.3001]


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.



Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [22]:
# создадим нужные переменные
std_avg = 100
size = 200000
mean_point = 525
mean_point_courses = 541.4
count_in_courses = 100

In [23]:
def get_p_value(mean_general, mu, std, size):
    std_error = std / numpy.sqrt(size)
    z = (mu - mean_general) / std_error
    return 1 - scipy.stats.norm.cdf(z)

In [24]:
print('{:.4f}'.format(get_p_value(mean_point, mean_point_courses, std_avg, count_in_courses)))

0.0505



Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [25]:
mean_point_courses_v2 = 541.5

In [26]:
print('{:.4f}'.format(get_p_value(mean_point, mean_point_courses_v2, std_avg, count_in_courses)))

0.0495
